In [12]:
import matplotlib.pyplot as plt
import pandas as pd
from census import Census
from us import states
from pathlib import Path
import geoviews as gv
import geoviews.feature as gf
import os


In [14]:
# Import U.S. Census API Key
from config import api_key

# Create an instance of the Census library
c = Census(
    api_key,
    year = 2020)

In [20]:
all_counties = []
for state in states.STATES:
    state_fips = state.fips
    county_data = c.acs5.state_county(('NAME', 'B19013_001E', 'B01003_001E'), state_fips, '*', year=2020)
    all_counties.append(pd.DataFrame(county_data))

#  Concatenate the county data for all states into a single DataFrame
all_counties_df = pd.concat(all_counties)

#  Rename the columns to make them more readable
all_counties_df = all_counties_df.rename(columns={
    'NAME': 'County',
    'B19013_001E': 'median_household_income',
    'B01003_001E': 'total_population'
})

#  Reset the index
all_counties_df = all_counties_df.reset_index(drop=True)

#  Print the DataFrame
all_counties_df.head()

,County,median_household_income,total_population,state,county
0,"Autauga County, Alabama",57982.0,55639.0,01,001
1,"Baldwin County, Alabama",61756.0,218289.0,01,003
2,"Barbour County, Alabama",34990.0,25026.0,01,005
3,"Bibb County, Alabama",51721.0,22374.0,01,007
4,"Blount County, Alabama",48922.0,57755.0,01,009


In [21]:
# rename 'state' as 'State
all_counties_df = all_counties_df.rename(columns={'state': 'State'})
# remove the 'county' column
all_counties_df = all_counties_df.drop(columns=['county'])
all_counties_df.head()

,County,median_household_income,total_population,State
0,"Autauga County, Alabama",57982.0,55639.0,01
1,"Baldwin County, Alabama",61756.0,218289.0,01
2,"Barbour County, Alabama",34990.0,25026.0,01
3,"Bibb County, Alabama",51721.0,22374.0,01
4,"Blount County, Alabama",48922.0,57755.0,01


In [22]:
# Convert the county column to string type
all_counties_df['County'] = all_counties_df['County'].astype(str)
# Split the county and state in the rows of the county column
all_counties_df[['County', 'State']] = all_counties_df['County'].str.split(',', expand=True)

# Rearrange the columns

all_counties_df = all_counties_df[['State', 'County', 'median_household_income', 'total_population']]


In [23]:
# remove spaces from the state column
all_counties_df['State'] = all_counties_df['State'].str.strip()
all_counties_df

,State,County,median_household_income,total_population
0,Alabama,Autauga County,57982.0,55639.0
1,Alabama,Baldwin County,61756.0,218289.0
2,Alabama,Barbour County,34990.0,25026.0
3,Alabama,Bibb County,51721.0,22374.0
4,Alabama,Blount County,48922.0,57755.0
...,...,...,...,...
3137,Wyoming,Hot Springs County,53398.0,4550.0
3138,Wyoming,Natrona County,62168.0,80067.0
3139,Wyoming,Platte County,57784.0,8572.0
3140,Wyoming,Sweetwater County,73384.0,43352.0


In [7]:
# write to csv for troubleshooting
# all_counties_df.to_csv("Resources/all_counties.csv", index=False)

In [24]:
# Read data from csv file
file = Path("Resources/Diabetes - 2020.csv")

diabetes_df = pd.read_csv(file)

diabetes_df = pd.DataFrame(diabetes_df[["State", "County", "Number"]])

diabetes_df = diabetes_df.rename(columns={"Number": "Diabetes Estimate"})

# remove rows with text in Diabetes Estimate column
diabetes_df = diabetes_df[diabetes_df["Diabetes Estimate"] != "Suppressed"]
diabetes_df = diabetes_df[diabetes_df["Diabetes Estimate"] != "No Data"]

# reset index
diabetes_df = diabetes_df.reset_index(drop=True)
diabetes_df

,State,County,Diabetes Estimate
0,Hawaii,Kalawao County,12
1,Texas,King County,17.6
2,Texas,Kenedy County,26.5
3,Nebraska,Arthur County,29.3
4,Nebraska,Blaine County,33.1
...,...,...,...
3135,California,Orange County,229477.5
3136,Arizona,Maricopa County,316604.6
3137,Texas,Harris County,344810.7
3138,Illinois,Cook County,356695.4


In [9]:
#write to csv for troubleshooting
# diabetes_df.to_csv('Resources/diabetes.csv', index=False)


In [25]:
# convert to strings
all_counties_df = all_counties_df.astype(str)
diabetes_df = diabetes_df.astype(str)

#Make county columns lowercase
all_counties_df['County'] = all_counties_df['County'].str.lower()
diabetes_df['County'] = diabetes_df['County'].str.lower()

# check data types for troubleshooting
# all_counties_df.info()
# diabetes_df.info()


In [26]:
merged_df = pd.merge(all_counties_df, diabetes_df, on=['State', 'County'], how='left')

# make the country column title format
merged_df['County'] = merged_df['County'].str.title() 
merged_df



,State,County,median_household_income,total_population,Diabetes Estimate
0,Alabama,Autauga County,57982.0,55639.0,3861.8
1,Alabama,Baldwin County,61756.0,218289.0,19353.8
2,Alabama,Barbour County,34990.0,25026.0,2200.8
3,Alabama,Bibb County,51721.0,22374.0,1708.3
4,Alabama,Blount County,48922.0,57755.0,5043.5
...,...,...,...,...,...
3137,Wyoming,Hot Springs County,53398.0,4550.0,357.7
3138,Wyoming,Natrona County,62168.0,80067.0,4726.8
3139,Wyoming,Platte County,57784.0,8572.0,595.1
3140,Wyoming,Sweetwater County,73384.0,43352.0,2368.3


In [188]:
# write to csv for troubleshooting
# merged_df.to_csv("Resources/merged_data.csv", index=False)